In [3]:
import pandas as pd
import matplotlib.pyplot as plt
# from statsmodels.tsa.stattools import coint, adfuller
# import statsmodels.api as sm


In [4]:
day_0 = pd.read_csv("prices_round_3_day_0.csv", sep=";")
day_1 = pd.read_csv("prices_round_3_day_1.csv", sep=";")
day_2 = pd.read_csv("prices_round_3_day_2.csv", sep=";")

all_days = pd.concat([day_0, day_1, day_2], ignore_index=True)
print(len(all_days.index))
all_days.head()

120000


,day,timestamp,product,bid_price_1,bid_volume_1,bid_price_2,bid_volume_2,bid_price_3,bid_volume_3,ask_price_1,ask_volume_1,ask_price_2,ask_volume_2,ask_price_3,ask_volume_3,mid_price,profit_and_loss
0,0,0,CHOCOLATE,7999,111,NaN,NaN,NaN,NaN,8001,111,NaN,NaN,NaN,NaN,8000.0,0.0
1,0,0,STRAWBERRIES,3999,210,NaN,NaN,NaN,NaN,4001,210,NaN,NaN,NaN,NaN,4000.0,0.0
2,0,0,ROSES,14999,72,NaN,NaN,NaN,NaN,15001,72,NaN,NaN,NaN,NaN,15000.0,0.0
3,0,0,GIFT_BASKET,71348,19,71347.0,24.0,NaN,NaN,71362,19,71363.0,24.0,NaN,NaN,71355.0,0.0
4,0,100,GIFT_BASKET,71344,1,71343.0,12.0,71342.0,20.0,71355,1,71356.0,12.0,71357.0,20.0,71349.5,0.0


In [5]:
choc_df = all_days[all_days["product"]=="CHOCOLATE"]
choc_df = choc_df.reset_index()
strawberry_df = all_days[all_days["product"]=="STRAWBERRIES"]
strawberry_df = strawberry_df.reset_index()
roses_df = all_days[all_days["product"]=="ROSES"]
roses_df = roses_df.reset_index()
gift_df = all_days[all_days["product"]=="GIFT_BASKET"]
gift_df = gift_df.reset_index()
# print(len(choc_df.index))
# choc_df.head()
# strawberry_df.head()
# roses_df.head()
# gift_df.head()

In [6]:
mid_prices = pd.DataFrame()
mid_prices["Chocolate"] = choc_df["mid_price"]
mid_prices["Strawberries"] = strawberry_df["mid_price"]
mid_prices["Roses"] = roses_df["mid_price"]
mid_prices["Gifts"] = gift_df["mid_price"]
mid_prices.corr()
# find that: 
# CHOCOLATE & GIFTS(1st) - 0.947, ROSES(2nd) - 0.826 have a STRONG correlation
# ROSES & GIFTS have a STRONG correlation (0.850)
# STRAWBERRIES & GIFTS have a MODERATE correlation (0.498)
# 
# Ideas:
# Strawberries appear to be relatively independent especially in comparison to chocolate and roses
# apart from gifts, all items are relatively independent of each other except for: chocolate & roses 

,Chocolate,Strawberries,Roses,Gifts
Chocolate,1.000000,0.273545,0.826358,0.947339
Strawberries,0.273545,1.000000,0.148667,0.498003
Roses,0.826358,0.148667,1.000000,0.850830
Gifts,0.947339,0.498003,0.850830,1.000000


In [7]:
# take the returns and check for stationarity
mid_prices['Chocolate_Returns'] = mid_prices['Chocolate'].pct_change().fillna(0)
mid_prices['Strawberries_Returns'] = mid_prices['Strawberries'].pct_change().fillna(0)
mid_prices['Roses_Returns'] = mid_prices['Roses'].pct_change().fillna(0)
mid_prices['Gifts_Returns'] = mid_prices['Gifts'].pct_change().fillna(0)
print(mid_prices.head())

   Chocolate  Strawberries    Roses    Gifts  Chocolate_Returns  \
0     8000.0        4000.0  15000.0  71355.0           0.000000   
1     8001.5        3999.5  15000.0  71349.5           0.000188   
2     8001.0        3999.5  15000.5  71347.0          -0.000062   
3     8002.5        3999.5  15000.0  71350.5           0.000187   
4     8002.0        3999.5  14997.5  71341.5          -0.000062   

   Strawberries_Returns  Roses_Returns  Gifts_Returns  
0              0.000000       0.000000       0.000000  
1             -0.000125       0.000000      -0.000077  
2              0.000000       0.000033      -0.000035  
3              0.000000      -0.000033       0.000049  
4              0.000000      -0.000167      -0.000126  


In [8]:
print(adfuller(mid_prices['Chocolate_Returns']))
print(adfuller(mid_prices['Strawberries_Returns']))
print(adfuller(mid_prices['Roses_Returns']))
print(adfuller(mid_prices['Gifts_Returns']))

#  all products are O(1) stationary

NameError: name 'adfuller' is not defined

In [ ]:
print(coint(mid_prices['Chocolate'], mid_prices['Roses']))
print(coint(mid_prices['Chocolate'], mid_prices['Strawberries']))
print(coint(mid_prices['Chocolate'], mid_prices['Gifts']))
print(coint(mid_prices['Roses'], mid_prices['Strawberries']))
print(coint(mid_prices['Roses'], mid_prices['Gifts']))
print(coint(mid_prices['Strawberries'], mid_prices['Gifts']))

### Cointegrated
5%:
- Roses & Gifts

10%:
- Chocolate & Gifts
- Roses & Strawberries

In [ ]:
for col in ["Chocolate", "Roses", "Strawberries", "Gifts"]:
    mid_prices[f"{col}_Rolling_Z_Score"] = (mid_prices[col] - mid_prices[col].rolling(30).mean())/mid_prices[col].rolling(30).std()
mid_prices.head()
print(mid_prices[30:60])

In [ ]:
plt.plot(mid_prices['Chocolate_Rolling_Z_Score'][:100], color="blue")
plt.title("CHOCOLATE_Z_SCORE")
plt.show()
plt.plot(mid_prices['Strawberries_Rolling_Z_Score'][:100], color="orange")
plt.title("STRAWBERRIE_Z_SCORE")
plt.show()
plt.plot(mid_prices['Roses_Rolling_Z_Score'][:100], color="yellow")
plt.title("ROSES_Z_SCORE")
plt.show()

In [ ]:
# pairs trading between roses and gifts
def find_coint(y0, y1, alpha=0.05):
    """
    y0 and y1 are the array-like 1d elements to compare for cointegration. Assuming y0 and y1 are I(1) stationary.

    Parameters:
    - y0: Array-like, 1d
    - y1: Array-like, 1d
    - alpha: the significance level is set at 0.5
    """
    # Fit OLS model
    model = sm.OLS(y0, y1)
    results = model.fit()
    
    # Get the estimated coefficient (beta) and standard error
    beta = results.params.iloc[0]
    std_err = results.bse.iloc[0]

    # Perform cointegration test
    _, p_value, _ = coint(y0, y1)

    print(p_value)
    
    # Check if there's cointegration
    cointegrated = p_value < alpha
    
    # Calculate the range
    lower_bound = beta - std_err
    upper_bound = beta + std_err
    
    return beta

In [ ]:
roses_gifts_spread = pd.DataFrame({
    "Roses": mid_prices['Roses'],
    "Gifts": mid_prices['Gifts']
}, index = mid_prices.index)


In [ ]:
# Get chocolate and gift spread
plt.plot(mid_prices["Chocolate"] - mid_prices["Gifts"], label="Chocolate/Gifts Spread")
plt.ylabel("Difference in Price")
plt.legend(loc="best")
plt.show()

In [ ]:
# CHOCOLATE & STRAWBERRIES
fig, ax1 = plt.subplots()
ax2 = ax1.twinx() 

choc_line = ax1.plot(choc_df["mid_price"], label="Chocolate Mid Price", color="BROWN")
strawberry_line = ax2.plot(strawberry_df["mid_price"], label="Strawberry Mid Price", color="PURPLE")

ax1.set_xlabel("Timestamp")
ax1.set_ylabel("Chocolate Mid Price")
ax2.set_ylabel("Strawberry Mid Price")

lns = choc_line + strawberry_line
labs = [l.get_label() for l in lns]
plt.legend(lns, labs, loc="best")
plt.show()

In [ ]:
# CHOCOLATE & ROSES
fig, ax1 = plt.subplots()
ax2 = ax1.twinx() 

choc_line = ax1.plot(choc_df["mid_price"], label="Chocolate Mid Price", color="BROWN")
roses_line = ax2.plot(roses_df["mid_price"], label="Roses Mid Price", color="RED")

ax1.set_xlabel("Timestamp")
ax1.set_ylabel("Chocolate Mid Price")
ax2.set_ylabel("Roses Mid Price")

lns = choc_line + roses_line
labs = [l.get_label() for l in lns]
plt.legend(lns, labs, loc="best")
plt.show()

In [ ]:
# CHOCOLATE & GIFTS
fig, ax1 = plt.subplots()
ax2 = ax1.twinx() 

choc_line = ax1.plot(choc_df["mid_price"], label="Chocolate Mid Price", color="BROWN")
gift_line = ax2.plot(gift_df["mid_price"], label="Gift Mid Price", color="BLUE")

ax1.set_xlabel("Timestamp")
ax1.set_ylabel("Chocolate Mid Price")
ax2.set_ylabel("Gift Mid Price")

lns = choc_line + gift_line
labs = [l.get_label() for l in lns]
plt.legend(lns, labs, loc="best")
plt.show()

In [ ]:
# STRAWBERRIES & ROSES
fig, ax1 = plt.subplots()
ax2 = ax1.twinx() 

strawberry_line = ax1.plot(strawberry_df["mid_price"], label="Strawberry Mid Price", color="PURPLE")
roses_line = ax2.plot(roses_df["mid_price"], label="Roses Mid Price", color="RED")

ax1.set_xlabel("Timestamp")
ax1.set_ylabel("Strawberry Mid Price")
ax2.set_ylabel("Roses Mid Price")

lns = strawberry_line + roses_line
labs = [l.get_label() for l in lns]
plt.legend(lns, labs, loc="best")
plt.show()

In [ ]:
# STRAWBERRIES & GIFTS
fig, ax1 = plt.subplots()
ax2 = ax1.twinx() 

strawberry_line = ax1.plot(strawberry_df["mid_price"], label="Strawberry Mid Price", color="PURPLE")
gifts_line = ax2.plot(gift_df["mid_price"], label="Gifts Mid Price", color="BLUE")

ax1.set_xlabel("Timestamp")
ax1.set_ylabel("Strawberry Mid Price")
ax2.set_ylabel("Gifts Mid Price")

lns = strawberry_line + gifts_line
labs = [l.get_label() for l in lns]
plt.legend(lns, labs, loc="best")
plt.show()

In [ ]:
# STRAWBERRIES & GIFTS
fig, ax1 = plt.subplots()
ax2 = ax1.twinx() 

roses_line = ax1.plot(roses_df["mid_price"], label="Roses Mid Price", color="RED")
gifts_line = ax2.plot(gift_df["mid_price"], label="Gifts Mid Price", color="BLUE")

ax1.set_xlabel("Timestamp")
ax1.set_ylabel("Roses Mid Price")
ax2.set_ylabel("Gifts Mid Price")

lns = roses_line + gifts_line
labs = [l.get_label() for l in lns]
plt.legend(lns, labs, loc="best")
plt.show()